# AI Chat Agent with Function Calling

This notebook demonstrates how to build an AI chat agent that can:
1. Maintain conversation history
2. Call functions/tools when needed
3. Process responses and continue conversations

## Setup and Imports

In [17]:
import openai,json

client = openai.OpenAI()
messages = []



## Define Functions and Tools

Here we define:
- **Functions**: Actual Python functions that perform tasks (e.g., get_weather)
- **FUNCTION_MAP**: Dictionary mapping function names to their implementations
- **TOOLS**: OpenAI-formatted descriptions of available functions

In [18]:
def get_weather(city):
    return f"The weather of {city} is sunny"

FUNCTION_MAP = {
    "get_weather": get_weather,
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather of a country",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to get the weather of"
                    }
                },
                "required": ["city"],
            }
        }
    }
]

## AI Response Processing Logic

The core logic that handles AI responses:
- **process_ai_response**: Checks if AI wants to call a function or just respond with text
- **call_ai**: Makes the API call to OpenAI with conversation history and available tools

When the AI decides to use a tool:
1. The tool call is added to the conversation
2. The function is executed locally
3. The result is added as a "tool" message
4. The AI is called again to process the result

In [19]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):

    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

## Interactive Chat Loop

The main chat interface:
- Continuously prompts for user input
- Adds user messages to conversation history
- Calls the AI and processes responses
- Type 'quit' or 'q' to exit

In [20]:
while True:
    message = input("You: ")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: what is the wether of spain
ChatCompletion(id='chatcmpl-C49tU7WJIfnl3bYM56PvkrLnGYOUq', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_IVxerVIR0paLWdylpSXwINQp', function=Function(arguments='{"city":"Spain"}', name='get_weather'), type='function')]))], created=1755107380, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=14, prompt_tokens=61, total_tokens=75, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
None


KeyboardInterrupt: Interrupted by user